임성열 Ph.D. 2025. 02.05, SKALA 수업 학습목적으로만 활용 바랍니다.

In [1]:
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
!pip install openai==1.55.3 httpx==0.27.2

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 KB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 KB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 20.7 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 774.0/774.0 KB 29.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.4/211.4 KB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.7/118.7 KB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 38.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 KB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 KB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 343.1/343.1 KB 29.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 39.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━

In [2]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [3]:
from crewai import Agent, Task, Crew

In [4]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

os.environ["OPENAI_API_KEY"] = api_key
os.environ["OPENAI_MODEL_NAME"] = 'gpt-4o-mini'


Agent(에이전트) 생성하기

Agent를 정의하고, role(역할), goal(목표), backstory(배경 설명)를 제공합니다.

LLM(대규모 언어 모델)은 롤플레잉을 할 때 더 나은 성능을 보이는 것으로 확인되었습니다.

### Agent: 기획자(Planner)

**참고**: _여러 문자열_ 사용의 이점:
```Python
varname = "텍스트의 첫 번째 줄"
          "텍스트의 두 번째 줄"
```

_삼중 따옴표 독스트링_ 사용과 비교했을 때:
```Python
varname = """텍스트의 첫 번째 줄
             텍스트의 두 번째 줄
          """
```
여러 문자열을 사용하면 공백과 줄바꿈 문자가 추가되는 것을 피할 수 있어, LLM에 전달할 때 더 나은 형식을 유지할 수 있습니다.

In [5]:
planner = Agent(
    role="콘텐츠 기획자",
    goal="{topic}에 대한 흥미로운 콘텐츠 기획",
    backstory="당신은 {topic}에 대한 글을 "
              "기획하는 일을 하고 있습니다. ",
    allow_delegation=False,
    verbose=True
)

### Agent: Writer

In [6]:
writer = Agent(
    role="콘텐츠 작성자",
    goal="{topic}에 대한"
         "의견을 포함한 글을 작성",
    backstory="당신은 {topic}에 대한"
              "의견을 포함한 기사를 작성하고 있습니다. ",
    allow_delegation=False,
    verbose=True
)

### Agent: Editor

In [7]:
editor = Agent(
    role="편집자",
    goal="주어진 기사 글을 올바른 표준 국어 형식 및 조직의 글 서식에 맞게 편집",
    backstory="당신은 콘텐츠 작성자로부터 "
              "기사 글을 받는 편집자입니다. ",
    allow_delegation=False,
    verbose=True
)

## Task(작업) 생성하기

- Task를 정의하고, `description`(설명), `expected_output`(예상 결과물), `agent`(수행 에이전트)를 제공합니다.


### Task: Plan

In [8]:
plan = Task(
    description=(
        "{topic}에 대한 최신 트렌드, 주요 인물, "
        "그리고 주목할 만한 뉴스를 우선순위화하기.\n"
    ),
    expected_output="개요, 독자층 분석, ",
    agent=planner,
)

### Task: Write

In [9]:
write = Task(
    description=(
        "1. 콘텐츠 기획을 활용하여 {topic}에 대한 "
            "매력적인 블로그 글 작성하기.\n"
    ),
    expected_output="각 섹션이 2~3개의 문단으로 구성된 글, ",
    agent=writer,
)

### Task: Edit

In [10]:
edit = Task(
    description=("주어진 블로그 글의 문법 오류를 검토하고 "
                 "브랜드의 톤앤매너와 일치하는지 확인하기."),
    expected_output="각 섹션이 2~3개의 문단으로 구성된, "
                    "지침에 맞추어진 준비가 완료된 마크다운 형식의 글",
    agent=editor
)

## Crew(팀) 생성하기

- Agent들로 구성된 팀을 생성합니다
- 해당 Agent들이 수행할 작업들을 전달합니다.
    - **참고**: *이 간단한 예시에서는* 작업들이 순차적으로 수행됩니다(즉, 서로 의존적임). 따라서 목록에서의 작업 _순서_가 _중요_합니다.
- `verbose=2`를 설정하면 실행의 모든 로그를 확인할 수 있습니다.


이 설정에서:
1. `planner`가 먼저 콘텐츠를 기획합니다
2. `writer`가 기획된 내용을 바탕으로 글을 작성합니다
3. `editor`가 최종적으로 작성된 글을 검토하고 편집합니다

In [11]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=2
)

## Running the Crew

In [12]:
result = crew.kickoff(inputs={"topic": "생성형 AI를 이용한 제약회사 경쟁력 제고 방안"})

 [DEBUG]: == Working Agent: 콘텐츠 기획자
 [INFO]: == Starting Task: 생성형 AI를 이용한 제약회사 경쟁력 제고 방안에 대한 최신 트렌드, 주요 인물, 그리고 주목할 만한 뉴스를 우선순위화하기.



> Entering new CrewAgentExecutor chain...
I now can give a great answer  
Final Answer: 

**개요**  
생성형 AI는 최근 제약업계에서 혁신과 경쟁력 제고의 중요한 도구로 부각되고 있습니다. 이 기술은 신약 개발, 임상 시험 최적화, 환자 맞춤형 치료 전략 수립, 그리고 마케팅 및 판매 전략 개선 등 여러 분야에서 활용되고 있습니다. 특히, 데이터 분석과 예측 모델링을 통해 제약회사들이 보다 빠르고 정확한 의사 결정을 내릴 수 있도록 도와줍니다. 본 글에서는 생성형 AI의 최신 트렌드, 주요 인물, 그리고 주목할 만한 뉴스들을 우선순위화하여 제약회사 경쟁력 제고 방안을 제시하고자 합니다.

**독자층 분석**  
이 콘텐츠는 제약회사 관계자, 연구개발(R&D) 부서 직원, 데이터 과학자, 마케팅 및 세일즈 팀, 그리고 제약 산업에 관심이 있는 일반 독자들을 대상으로 합니다. 특히, 제약업계의 최신 기술 트렌드에 관심이 있는 전문가와 의사결정자는 이 글에서 유용한 인사이트를 얻을 수 있을 것입니다.

**최신 트렌드**  
1. **신약 발견 및 개발 가속화**  
   - 생성형 AI는 화합물의 구조를 설계하고, 예상되는 생물학적 활성을 예측하여 신약 발견 과정을 혁신하고 있습니다. 예를 들어, DeepMind의 AlphaFold는 단백질 구조를 예측하는 데 있어 획기적인 발전을 이뤘습니다.

2. **임상 시험 효율성 향상**  
   - 환자 선정, 시험 설계, 데이터 분석 과정에서 AI를 활용하여 임상 시험의 성공률을 높이고 기간을 단축할 수 있습니다. IBM Watson Health는 이러한 솔루션을 제공하여 임상 시험의 품질을 높이고 있습니

- Display the results of your execution as markdown in the notebook.

In [13]:
from IPython.display import Markdown
Markdown(result)

# 생성형 AI를 이용한 제약회사 경쟁력 제고 방안

## 개요  
최근 제약업계에서 생성형 AI는 혁신과 경쟁력 제고의 중추적인 도구로 자리 잡고 있습니다. 이 기술은 신약 개발, 임상 시험 최적화, 환자 맞춤형 치료 전략 수립, 마케팅 및 판매 전략 개선 등 다양한 분야에서 활용되고 있으며, 데이터 분석과 예측 모델링을 통해 제약회사들이 보다 효율적이고 정교한 의사 결정을 내릴 수 있도록 지원하고 있습니다. 본 글에서는 생성형 AI의 최신 트렌드, 주요 인물, 그리고 주목할 만한 뉴스들을 중심으로 제약회사 경쟁력 제고 방안을 제시하고자 합니다.

## 최신 트렌드  
첫 번째로, 생성형 AI는 신약 발견 및 개발을 가속화하는 데 큰 기여를 하고 있습니다. 예를 들어, DeepMind의 AlphaFold는 단백질 구조 예측에 있어 혁신적인 발전을 이루어, 화합물의 구조 설계와 생물학적 활성을 예측하는 데 도움을 줍니다. 이는 제약회사들이 새로운 치료제를 보다 빠르게 개발할 수 있도록 지원하며, 시장에서의 경쟁 우위를 강화하는 데 기여합니다.

두 번째로, 임상 시험의 효율성을 향상시키는 데 있어 생성형 AI는 필수적인 역할을 하고 있습니다. 환자 선정, 시험 설계, 데이터 분석 과정에서 AI를 활용하여 임상 시험의 성공률을 높이고 기간을 단축할 수 있습니다. IBM Watson Health와 같은 기업들은 이러한 솔루션을 제공하여 임상 시험의 품질을 개선하고, 결과적으로 제약 회사의 시간과 비용을 절감하는 데 도움을 주고 있습니다.

세 번째로, 환자 맞춤형 치료의 발전도 주목할 만합니다. 생성형 AI는 유전자 데이터 및 환자의 병력 정보를 분석하여 개인 맞춤형 치료법을 개발하는 데 기여하고 있습니다. 이를 통해 환자들의 치료 반응을 극대화하고 부작용을 최소화할 수 있는 가능성이 열리고 있으며, 이는 환자 만족도를 높이고 치료 성과를 개선하는 데 중요한 요소로 작용하고 있습니다.

## 주요 인물  
AI 분야의 선두주자인 Andrew Ng는 제약 산업에 AI 기술을 도입하는 데 기여하며, 이 기술의 발전 가능성을 널리 알리고 있습니다. 또한, Fei-Fei Li는 AI 윤리와 데이터 사용에 대한 논의를 이끌며 제약 분야에서의 책임 있는 AI 사용을 강조하고 있습니다. Demis Hassabis, DeepMind의 CEO는 생명과학 분야에서 AI 기술의 선도적인 활용을 통해 제약 산업의 혁신을 이끌고 있습니다.

## 주목할 만한 뉴스  
Moderna는 AI 기술을 활용하여 mRNA 기반 백신의 개발 속도를 높이고 있으며, 이러한 협업은 혁신적인 치료법을 지속적으로 선보이는 데 기여하고 있습니다. 또한, AI 기반 신약 개발 스타트업들이 증가함에 따라 제약업계의 경쟁 구조가 변화하고 있습니다. 이러한 변화는 새로운 아이디어와 접근 방식을 통해 제약 산업의 발전을 이끌어 낼 것입니다. 마지막으로, 미국 FDA는 AI 및 머신러닝을 활용한 의료기기와 치료법에 대한 규제를 강화하고 있으며, 이는 안전하고 효과적인 AI 활용을 위한 중요한 기초를 마련하고 있습니다.

## 결론  
생성형 AI는 제약회사의 경쟁력을 높이는 데 있어 중요한 역할을 합니다. 이를 통해 혁신적인 신약 개발, 환자 맞춤형 치료, 효율적인 임상 시험 운영 및 효과적인 마케팅 전략 수립이 가능해집니다. 제약업계 관계자들은 이러한 최신 트렌드와 인사이트를 바탕으로 경쟁력을 제고할 수 있는 전략을 모색해야 하며, 미래의 제약 산업에서 성공하기 위해서는 생성형 AI의 도입과 활용이 필수적입니다.